#UCI_HAR_load_dataset.ipynb. 
Loads the [UCI HAR (Human Activity Recognition Using Smartphones)](https://archive.ics.uci.edu/ml/datasets/human+activity+recognition+using+smartphones#) dataset from the Internet repository and converts the acceleration data into numpy arrays while adhering to the general format of the [Keras MNIST load_data function](https://keras.io/api/datasets/mnist/#load_data-function).

This uses the original dataset which already has significant pre-processing such as separation into train/test and parsing into a 2.56 second sliding window with 50% overlap.   The acceleration imported is the body accel (total accel less 1g gravity) from train/Inertial Signals/body_acc_x_train.txt.

So this function is more of a format converter from text to numpy.  It does not have the same level of adjustable parameters as some of the other loaders.

The train/test split is by subject as per the dataset documentation.

Example usage:

    x_train, y_train, x_test, y_test = uci_har_load_dataset()

Per the release notes use of this dataset in publications must be acknowledged by referencing the following publication   
> Davide Anguita, Alessandro Ghio, Luca Oneto, Xavier Parra and Jorge L. Reyes-Ortiz. A Public Domain Dataset for Human Activity Recognition Using Smartphones. 21th European Symposium on Artificial Neural Networks, Computational Intelligence and Machine Learning, ESANN 2013. Bruges, Belgium 24-26 April 2013.   

Developed and tested using colab.research.google.com  
To save as .py version use File > Download .py

Author:  [Lee B. Hinkle](https://userweb.cs.txstate.edu/~lbh31/), [IMICS Lab](https://imics.wp.txstate.edu/), Texas State University, 2021

<a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-sa/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">Creative Commons Attribution-ShareAlike 4.0 International License</a>.

TODOs:
* Would be better to have specific subjects in validation vs stratify.


In [8]:
import os
import shutil #https://docs.python.org/3/library/shutil.html
from shutil import unpack_archive # to unzip
import requests #for downloading zip file
import numpy as np
from tabulate import tabulate # for verbose tables, showing data
from tensorflow.keras.utils import to_categorical # for one-hot encoding
from sklearn.model_selection import train_test_split

In [3]:
#credit https://stackoverflow.com/questions/9419162/download-returned-zip-file-from-url
#many other methods I tried failed to download the file properly
def download_url(url, save_path, chunk_size=128):
    """downloads file from url and saves it at save_path"""
    r = requests.get(url, stream=True)
    with open(save_path, 'wb') as fd:
        for chunk in r.iter_content(chunk_size=chunk_size):
            fd.write(chunk)

In [10]:
def uci_har_load_dataset(
    verbose = True,
    incl_xyz_accel = False, #include component accel_x/y/z in ____X data
    incl_rms_accel = True, #add rms value (total accel) of accel_x/y/z in ____X data
    incl_val_group = False, # see note below. UCI HAR - already split train/test
    one_hot_encode = True
    ):
    """processes UCI HAR zip file into numpy arrays, returns x_train, y_train, x_test, y_test"""
    #Download and unzip original dataset
    if (not os.path.isfile('./UCI_HAR_Dataset.zip')):
        print("Downloading UCI_HAR_Dataset.zip file")
        download_url('https://archive.ics.uci.edu/ml/machine-learning-databases/00240/UCI%20HAR%20Dataset.zip','./UCI_HAR_Dataset.zip')
    if (not os.path.isdir('/content/UCI HAR Dataset')):
        print("Unzipping UCI_HAR_Dataset.zip file")
        shutil.unpack_archive('./UCI_HAR_Dataset.zip','.','zip')
    
    #Load .txt files as numpy ndarrays
    path_in = '/content/UCI HAR Dataset/'

    body_acc_x_train = np.loadtxt('/content/UCI HAR Dataset/train/Inertial Signals/body_acc_x_train.txt')
    body_acc_y_train = np.loadtxt('/content/UCI HAR Dataset/train/Inertial Signals/body_acc_y_train.txt')
    body_acc_z_train = np.loadtxt('/content/UCI HAR Dataset/train/Inertial Signals/body_acc_z_train.txt')
    y_train = np.loadtxt('/content/UCI HAR Dataset/train/y_train.txt')

    body_acc_x_test = np.loadtxt('/content/UCI HAR Dataset/test/Inertial Signals/body_acc_x_test.txt')
    body_acc_y_test = np.loadtxt('/content/UCI HAR Dataset/test/Inertial Signals/body_acc_y_test.txt')
    body_acc_z_test = np.loadtxt('/content/UCI HAR Dataset/test/Inertial Signals/body_acc_z_test.txt')
    y_test = np.loadtxt('/content/UCI HAR Dataset/test/y_test.txt')

    #reshape to stack component accel and calculate rms accel
    acc_train = np.dstack((body_acc_x_train,body_acc_y_train,body_acc_z_train))
    acc_test = np.dstack((body_acc_x_test,body_acc_y_test,body_acc_z_test))
    if (incl_rms_accel):
        rms_accel_train = np.sqrt((acc_train[:,:,0]**2) + (acc_train[:,:,1]**2) + (acc_train[:,:,2]**2))
        rms_accel_test = np.sqrt((acc_test[:,:,0]**2) + (acc_test[:,:,1]**2) + (acc_test[:,:,2]**2))
        x_train = np.dstack((acc_train,rms_accel_train))
        x_test = np.dstack((acc_test,rms_accel_test))
    #remove component accel if needed
    if (not incl_xyz_accel):
        x_train = np.delete(x_train, [0,1,2], 2)
        x_test = np.delete(x_test, [0,1,2], 2)
    if (one_hot_encode):
        y_train = y_train - 1 #original was 1 - 6
        y_test = y_test -1
        y_train = to_categorical(y_train, num_classes=6)
        y_test = to_categorical(y_test, num_classes=6)

    if (incl_val_group):
        print('\nWarning: UCI HAR is already split into train/test')
        print('The validation group is generated using sklearn stratify on train')
        print('It is not subject independent - confirm accuracy with test set')
        x_train, x_validation, y_train, y_validation = train_test_split(x_train, y_train, test_size=0.25, random_state=42, stratify=y_train)
        return x_train, y_train, x_validation, y_validation, x_test, y_test
    else:
        if(verbose):
            headers = ("Reshaped data","shape", "object type", "data type")
            mydata = [("x_train:", x_train.shape, type(x_train), x_train.dtype),
                    ("y_train:", y_train.shape ,type(y_train), y_train.dtype),
                    ("x_test:", x_test.shape, type(x_test), x_test.dtype),
                    ("y_test:", y_test.shape ,type(y_test), y_test.dtype)]
            print(tabulate(mydata, headers=headers))

        return x_train, y_train, x_test, y_test

In [11]:
if __name__ == "__main__":
    print("Downloading and processing UCI HAR dataset")
    x_train, y_train, x_test, y_test = uci_har_load_dataset()
    print("\nUCI HAR returned arrays:")
    print("x_train shape ",x_train.shape," y_train shape ", y_train.shape)
    print("x_test shape  ",x_test.shape," y_test shape  ",y_test.shape)

    x_train, y_train, x_validation, y_validation, x_test, y_test = uci_har_load_dataset(incl_val_group=True)
    print("\nUCI HAR returned arrays with validation group:")
    print("x_train shape ",x_train.shape," y_train shape ", y_train.shape)
    print("x_validation shape ",x_validation.shape," y_validation shape ", y_validation.shape)
    print("x_test shape  ",x_test.shape," y_test shape  ",y_test.shape)

Reshaped data    shape           object type              data type
---------------  --------------  -----------------------  -----------
x_train:         (7352, 128, 1)  <class 'numpy.ndarray'>  float64
y_train:         (7352, 6)       <class 'numpy.ndarray'>  float32
x_test:          (2947, 128, 1)  <class 'numpy.ndarray'>  float64
y_test:          (2947, 6)       <class 'numpy.ndarray'>  float32

UCI HAR returned arrays:
x_train shape  (7352, 128, 1)  y_train shape  (7352, 6)
x_test shape   (2947, 128, 1)  y_test shape   (2947, 6)

The validation group is generated using sklearn stratify on train
It is not subject independent - confirm accuracy with test set

UCI HAR returned arrays with validation group:
x_train shape  (5514, 128, 1)  y_train shape  (5514, 6)
x_validation shape  (1838, 128, 1)  y_validation shape  (1838, 6)
x_test shape   (2947, 128, 1)  y_test shape   (2947, 6)
